
## Calibration tests
### Pose2Sim calibration.py functions
### KaggleHub dataset "danielwe14/stereocamera-chessboard-pictures"

Auteurs : Léa Drolet-Roy

Création : 2025-02-14

Dernière modification :

In [ ]:
import numpy as np
import cv2

In [ ]:
h, w = image_size
camera_matrix_init = np.array([
    [focal_length, 0, w / 2],
    [0, focal_length, h / 2],
    [0, 0, 1]], dtype=np.float32)

ret, C, S, D, K = Board.calibrate_intrinsics(calib_dir, cams, image_size, camera_matrix_init)

K = np.array(K)
np.savez('Intrinsics_K_trampo.npz', K)
D = np.array(D)
np.savez('Intrinsics_D_trampo.npz', D)

    

In [ ]:
## PSO for extrinsic parameters (with Bundle adjustment)
# Constants
num_cameras = 7
num_params_per_cam = 6
total_params = num_cameras * num_params_per_cam
n_particles = 60  # Number of particles
options = {'c1': 1.5, 'c2': 1.5, 'w': 0.7} #{'c1': 2.05, 'c2': 2.05, 'w': 0.729} # PSO Hyperparameters

projMat = np.load('Extrinsics_optimized_3.npz')['arr_0']

init_params = []
for mat in projMat:
    rvec, _ = cv2.Rodrigues(mat[0:3,0:3])
    rvec = rvec.squeeze()
    params = [rvec[0], rvec[1], rvec[2], mat[0,3], mat[1,3], mat[2,3]]
    init_params.extend(params)

init_params = np.array(init_params, dtype=np.float64)
init_pos = np.tile(init_params, (n_particles,1))
print(init_params.shape)
print(init_pos.shape)

# Define parameter bounds
lower_bounds = [-2*np.pi, -2*np.pi, -2*np.pi, -5000, -5000, -5000]
upper_bounds = [2*np.pi, 2*np.pi, 2*np.pi, 5000, 5000, 6000]

param_bounds = (np.tile(lower_bounds, num_cameras), np.tile(upper_bounds, num_cameras))
print(param_bounds[0].shape)

print(np.all(param_bounds[0] <= init_pos[0]))
print(np.all(init_pos[0] <= param_bounds[1]))